In [1]:
#First import companies dataset
from pymongo import MongoClient
import functions as fns
import pandas as pd
import numpy as np
import folium
import json
import requests
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import os
from geopy.distance import distance
conn = MongoClient("localhost:27017")
conn.list_database_names()


['admin', 'companies', 'config', 'ironhack', 'local']

# Querying MongoDB

In [3]:
#Chose database to workwith
db = conn.get_database("companies")

In [4]:
collection = db.get_collection("companies")

In [5]:
collection.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'permalink': 'abc3',
 'crunchbase_url': 'http://www.crunchbase.com/company/adventnet',
 'homepage_url': 'http://adventnet.com',
 'blog_url': '',
 'blog_feed_url': '',
 'twitter_username': 'manageengine',
 'category_code': 'enterprise',
 'number_of_employees': 600,
 'founded_year': 1996,
 'deadpooled_year': 2,
 'tag_list': '',
 'alias_list': 'Zoho ManageEngine ',
 'email_address': 'pr@adventnet.com',
 'phone_number': '925-924-9500',
 'description': 'Server Management Software',
 'created_at': datetime.datetime(2007, 5, 25, 19, 24, 22),
 'updated_at': 'Wed Oct 31 18:26:09 UTC 2012',
 'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
 'image': {'avail

In [6]:
collection.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

**Our objective:** To find the ideal location for a new comoany with the following structure:

20 Designers

5 UI/UX Engineers

10 Frontend Developers

15 Data Engineers

5 Backend Developers

20 Account Managers

1 Maintenance guy that loves basketball

10 Executives

1 CEO/President

**With these constraints:**

Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

30% of the company staff have at least 1 child.

Developers like to be near successful tech startups that have raised at least 1 Million dollars.

Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
Account managers need to travel a lot.

Everyone in the company is between 25 and 40, give them some place to go party.

The CEO is vegan.

If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.

The office dog— ~"Pepe"~ Dobby needs a hairdresser every month. Ensure there's one not too far away.



Constraints from mongodb:
1, 3
How will these be tackled?:
For 1: Find all design companies from companies database
For 3: Find all startups (we will define as being created in the past 3 years) which have raised over 1 million dollars.

In [7]:
#Find distint company codes and identify those related with design
collection.distinct("category_code")

[None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web']

In [8]:
design_companies = collection.find({"category_code": {"$nin": ["design", "software", "web","games_video", "advertising", "ecommerce", "analytics","photo_video"]}}, {"name":1, "category_code": 1, "_id":0})
len(list(design_companies))

9486

In [9]:
#After querying we find that the newest companies on this dataset were founded
#in 2013 so we will find all companies founded after 2008
startups = collection.find({"founded_year": {"$gt": 2008}}, {"name":1, "founded_year": 1, "_id":0})
len(list(startups))

833

In [10]:
#From those startups find the ones which were tech related and had raised over
#1 million dollars such that they satisfy the previous conditions
succesful_startups = collection.find({"$and": [
        {"founded_year": {"$gt":2008}},
        {"category_code": {"$nin": ["design", "software", "web","games_video", "advertising", "ecommerce", "analytics","photo_video"]}},
        {"total_money_raised": {"$regex":".*M"}}
    ]},
    {"name": 1, "founded_year": 1 ,"total_money_raised": 1, "_id": 0})
len(list(succesful_startups))

64

In [11]:
#Create dataframe from the found companies
df = pd.DataFrame(collection.find({"$and": [
        {"founded_year": {"$gt":2008}},
        {"category_code": {"$nin": ["design", "software", "web","games_video", "advertising", "ecommerce", "analytics","photo_video"]}},
        {"total_money_raised": {"$regex":".*M"}}
    ]},
    {"name": 1, "founded_year": 1 ,"total_money_raised": 1, "_id": 0, "offices.longitude":1, "offices.latitude":1}))


correct_addresses = df.explode("offices")


In [13]:
#Create new columns for the coordinates of offices and corresponding geopoints
correct_addresses["lat"] = correct_addresses["offices"].apply(fns.get_lat)
correct_addresses["long"] = correct_addresses["offices"].apply(fns.get_long)
correct_addresses["geopoint"] = correct_addresses["offices"].apply(fns.create_geopoint_from_office)

#Drop rows without location so we can plot on map
correct_addresses = correct_addresses.dropna(how='any',axis=0)

correct_addresses.head(20)

,name,founded_year,total_money_raised,offices,lat,long,geopoint
1,PeekYou,2012,$1.83M,"{'latitude': 40.757929, 'longitude': -73.985506}",40.757929,-73.985506,"{'type': 'Point', 'coordinates': [40.757929, -..."
2,GENWI,2010,$7.1M,"{'latitude': 33.8171, 'longitude': -111.9035}",33.817100,-111.903500,"{'type': 'Point', 'coordinates': [33.8171, -11..."
4,Needium,2010,$1M,"{'latitude': 45.5049305, 'longitude': -73.5681...",45.504931,-73.568163,"{'type': 'Point', 'coordinates': [45.5049305, ..."
6,Widgetbox,2012,$14.5M,"{'latitude': 37.798853, 'longitude': -122.398599}",37.798853,-122.398599,"{'type': 'Point', 'coordinates': [37.798853, -..."
7,Skydeck,2012,$4M,"{'latitude': 37.5645375, 'longitude': -122.325...",37.564538,-122.325470,"{'type': 'Point', 'coordinates': [37.5645375, ..."
8,Moblica,2011,$6M,"{'latitude': 32.0554, 'longitude': 34.7595}",32.055400,34.759500,"{'type': 'Point', 'coordinates': [32.0554, 34...."
10,GetYourGuide,2009,$16M,"{'latitude': 47.3912899, 'longitude': 8.5082891}",47.391290,8.508289,"{'type': 'Point', 'coordinates': [47.3912899, ..."
11,travelmob,2011,$1M,"{'latitude': 21.303049, 'longitude': -157.78907}",21.303049,-157.789070,"{'type': 'Point', 'coordinates': [21.303049, -..."
12,UsingMiles,2010,$3.27M,"{'latitude': 39.749671, 'longitude': -104.9998...",39.749671,-104.999845,"{'type': 'Point', 'coordinates': [39.749671, -..."
13,travelmob,2011,$1M,"{'latitude': 21.303049, 'longitude': -157.78907}",21.303049,-157.789070,"{'type': 'Point', 'coordinates': [21.303049, -..."


# Plot the offices found

In [14]:
map_2 = Map(location=[40.428990,-3.681209],zoom_start=1)

map_2

In [15]:
#Plot located offices
for i,row in correct_addresses.iterrows():
    name = {
        "location" : [row["lat"], row["long"]],
        "tooltip" : row["name"]
    }
    
    Marker (**name).add_to(map_2)
map_2

Now lets start to work with the rest of the requirements, for which we will use Foursquares API to get data.

The "model" which will I will use is one which will rank all office locations plotted above based on the ratio between the distance between for an example a starbucks and the maximum distance an employee will commute to get there (1km in this case) in such a way that after running this test for all requirements the location with the least score will be the one we chose as the "ideal" location for our company


# Using FourSquares API

In [16]:
load_dotenv()

True

In [17]:
#Lets set up variables to make API calls from Foursquare
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")
url = 'https://api.foursquare.com/v2/venues/explore'
casa = {'type': 'Point', 'coordinates': [40.519128, -3.647405]}

In [18]:
#Create function which will get a location and the name of a place and will
#Find the distance between them
def get_place_distance(geopoint, place):
    try:
        url = 'https://api.foursquare.com/v2/venues/explore'
        parametros = { "client_id" : tok1,
              "client_secret" : tok2,
              "v" : "20180323",
              "ll": f"{geopoint.get('coordinates')[0]},{geopoint.get('coordinates')[1]}",
              "query": f"{place}",
              "limit" : 1
        }
        resp = requests.get(url=url, params=parametros)
        data = json.loads(resp.text)
        lat = data['response']['groups'][0]["items"][0]["venue"]["location"]["lat"]
        long = data['response']['groups'][0]["items"][0]["venue"]["location"]["lng"]
        coords = (lat,long)
        office_coords = (geopoint.get('coordinates')[0], geopoint.get('coordinates')[1])
        d = distance(coords, office_coords).m #distance in metres
        return d
    except IndexError:
        return None

In [19]:
#Add columns for distance between aminities and office
correct_addresses['starb_dist'] = correct_addresses.apply(lambda x: get_place_distance(x['geopoint'], "starbucks"), axis=1)
correct_addresses['airport_distance'] = correct_addresses.apply(lambda x: get_place_distance(x['geopoint'], "airport"), axis=1)
correct_addresses['school_distance'] = correct_addresses.apply(lambda x: get_place_distance(x['geopoint'], "school"), axis=1)
correct_addresses['nightclub_distance'] = correct_addresses.apply(lambda x: get_place_distance(x['geopoint'], "club"), axis=1)
correct_addresses['vegan_distance'] = correct_addresses.apply(lambda x: get_place_distance(x['geopoint'], "vegan"), axis=1)
correct_addresses['basketball_distance'] = correct_addresses.apply(lambda x: get_place_distance(x['geopoint'], "basketball stadium"), axis=1)
correct_addresses['dobby_distance'] = correct_addresses.apply(lambda x: get_place_distance(x['geopoint'], "hairdresser"), axis=1)

correct_addresses.head(30)

,name,founded_year,total_money_raised,offices,lat,long,geopoint,starb_dist,airport_distance,school_distance,nightclub_distance,vegan_distance,basketball_distance,dobby_distance
1,PeekYou,2012,$1.83M,"{'latitude': 40.757929, 'longitude': -73.985506}",40.757929,-73.985506,"{'type': 'Point', 'coordinates': [40.757929, -...",163.614942,135.050617,383.837150,280.265622,346.476253,1046.783033,810.070203
2,GENWI,2010,$7.1M,"{'latitude': 33.8171, 'longitude': -111.9035}",33.817100,-111.903500,"{'type': 'Point', 'coordinates': [33.8171, -11...",1868.780687,105.472458,6002.430008,3093.711361,16088.831001,5508.727017,57190.867355
4,Needium,2010,$1M,"{'latitude': 45.5049305, 'longitude': -73.5681...",45.504931,-73.568163,"{'type': 'Point', 'coordinates': [45.5049305, ...",499.188440,15144.766081,1444.985426,376.579897,259.574524,1024.634985,998.042974
6,Widgetbox,2012,$14.5M,"{'latitude': 37.798853, 'longitude': -122.398599}",37.798853,-122.398599,"{'type': 'Point', 'coordinates': [37.798853, -...",497.485029,20241.330111,412.308879,246.659469,662.939923,3576.697169,1174.137592
7,Skydeck,2012,$4M,"{'latitude': 37.5645375, 'longitude': -122.325...",37.564538,-122.325470,"{'type': 'Point', 'coordinates': [37.5645375, ...",224.052751,7947.052661,183.908315,248.547389,618.594894,NaN,4220.821867
8,Moblica,2011,$6M,"{'latitude': 32.0554, 'longitude': 34.7595}",32.055400,34.759500,"{'type': 'Point', 'coordinates': [32.0554, 34....",1916.687611,12313.586085,519.662631,50.453277,321.938603,3070.573435,3185.029081
10,GetYourGuide,2009,$16M,"{'latitude': 47.3912899, 'longitude': 8.5082891}",47.391290,8.508289,"{'type': 'Point', 'coordinates': [47.3912899, ...",607.509153,7954.621423,545.171609,1146.055835,1787.125399,NaN,2191.376097
11,travelmob,2011,$1M,"{'latitude': 21.303049, 'longitude': -157.78907}",21.303049,-157.789070,"{'type': 'Point', 'coordinates': [21.303049, -...",2340.134374,13938.870826,4154.052746,6948.374482,3742.884403,2491.391562,6429.024308
12,UsingMiles,2010,$3.27M,"{'latitude': 39.749671, 'longitude': -104.9998...",39.749671,-104.999845,"{'type': 'Point', 'coordinates': [39.749671, -...",94.154159,30037.913085,237.936137,108.794356,583.276063,672.587646,27840.619599
13,travelmob,2011,$1M,"{'latitude': 21.303049, 'longitude': -157.78907}",21.303049,-157.789070,"{'type': 'Point', 'coordinates': [21.303049, -...",2340.134374,13938.870826,4154.052746,6948.374482,3742.884403,2491.391562,6429.024308


# Rate each location with our model

In [ ]:
#This didnt work but I left the code to show my intention to not write bad code
# :)
#def overall_rater(row):
#    starb_score = single_rater(row["starb_dist"],300)
#    airp_score = single_rater(row["airport_distance"],15000)
#    school_score = single_rater(row["school_distance"],4000)
#    club_score = single_rater(row["nightclub_distance"],1000)
#    vegan_score = single_rater(row["vegan_distance"],300)
#    bbal_score = single_rater(row["basketball_distance"],10000)
#    dobby_score = single_rater(row["dobby_distance"],4000)
    
#    formula = starb_score + airp_score + school_score + club_score + vegan_score + bbal_score + single_rater
    
#    return formula

In [21]:
#Assign score based on our model
correct_addresses['location_score'] = correct_addresses.apply(fns.overall_rater, axis = 1)
correct_addresses.head(30)

,name,founded_year,total_money_raised,offices,lat,long,geopoint,starb_dist,airport_distance,school_distance,nightclub_distance,vegan_distance,basketball_distance,dobby_distance,location_score
1,PeekYou,2012,$1.83M,"{'latitude': 40.757929, 'longitude': -73.985506}",40.757929,-73.985506,"{'type': 'Point', 'coordinates': [40.757929, -...",163.614942,135.050617,383.837150,280.265622,346.476253,1046.783033,810.070203,3.237807
2,GENWI,2010,$7.1M,"{'latitude': 33.8171, 'longitude': -111.9035}",33.817100,-111.903500,"{'type': 'Point', 'coordinates': [33.8171, -11...",1868.780687,105.472458,6002.430008,3093.711361,16088.831001,5508.727017,57190.867355,10.557904
4,Needium,2010,$1M,"{'latitude': 45.5049305, 'longitude': -73.5681...",45.504931,-73.568163,"{'type': 'Point', 'coordinates': [45.5049305, ...",499.188440,15144.766081,1444.985426,376.579897,259.574524,1024.634985,998.042974,5.955049
6,Widgetbox,2012,$14.5M,"{'latitude': 37.798853, 'longitude': -122.398599}",37.798853,-122.398599,"{'type': 'Point', 'coordinates': [37.798853, -...",497.485029,20241.330111,412.308879,246.659469,662.939923,3576.697169,1174.137592,7.000941
7,Skydeck,2012,$4M,"{'latitude': 37.5645375, 'longitude': -122.325...",37.564538,-122.325470,"{'type': 'Point', 'coordinates': [37.5645375, ...",224.052751,7947.052661,183.908315,248.547389,618.594894,NaN,4220.821867,7.571170
8,Moblica,2011,$6M,"{'latitude': 32.0554, 'longitude': 34.7595}",32.055400,34.759500,"{'type': 'Point', 'coordinates': [32.0554, 34....",1916.687611,12313.586085,519.662631,50.453277,321.938603,3070.573435,3185.029081,6.104589
10,GetYourGuide,2009,$16M,"{'latitude': 47.3912899, 'longitude': 8.5082891}",47.391290,8.508289,"{'type': 'Point', 'coordinates': [47.3912899, ...",607.509153,7954.621423,545.171609,1146.055835,1787.125399,NaN,2191.376097,9.214445
11,travelmob,2011,$1M,"{'latitude': 21.303049, 'longitude': -157.78907}",21.303049,-157.789070,"{'type': 'Point', 'coordinates': [21.303049, -...",2340.134374,13938.870826,4154.052746,6948.374482,3742.884403,2491.391562,6429.024308,11.178397
12,UsingMiles,2010,$3.27M,"{'latitude': 39.749671, 'longitude': -104.9998...",39.749671,-104.999845,"{'type': 'Point', 'coordinates': [39.749671, -...",94.154159,30037.913085,237.936137,108.794356,583.276063,672.587646,27840.619599,6.549384
13,travelmob,2011,$1M,"{'latitude': 21.303049, 'longitude': -157.78907}",21.303049,-157.789070,"{'type': 'Point', 'coordinates': [21.303049, -...",2340.134374,13938.870826,4154.052746,6948.374482,3742.884403,2491.391562,6429.024308,11.178397


In [30]:
#Now lets sort our table to find the ideal location according to our model:
correct_addresses.sort_values(by='location_score', ascending=True,inplace = True)
df.to_csv('sorted_companies.csv')

In [23]:
#lets find the best location
best_office = correct_addresses.iloc[0]
print(best_office)

name                                                            oneforty
founded_year                                                        2009
total_money_raised                                                $2.35M
offices                 {'latitude': 42.364014, 'longitude': -71.101375}
lat                                                               42.364
long                                                            -71.1014
geopoint               {'type': 'Point', 'coordinates': [42.364014, -...
starb_dist                                                       257.171
airport_distance                                                 1211.16
school_distance                                                  70.6107
nightclub_distance                                               253.232
vegan_distance                                                   44.7227
basketball_distance                                              3241.12
dobby_distance                                     

# Plot the ideal office location

In [24]:
#display best location
map_3 = Map(location=[40.428990,-3.681209],zoom_start=1)

best_location = Marker(location=[best_office["lat"],best_office["long"]], tooltip = "Best location")
best_location.add_to(map_3)
map_3

In [25]:
#display corresponding amenities
def get_place_coords(geopoint, place):
    try:
        url = 'https://api.foursquare.com/v2/venues/explore'
        parametros = { "client_id" : tok1,
              "client_secret" : tok2,
              "v" : "20180323",
              "ll": f"{geopoint.get('coordinates')[0]},{geopoint.get('coordinates')[1]}",
              "query": f"{place}",
              "limit" : 1
        }
        resp = requests.get(url=url, params=parametros)
        data = json.loads(resp.text)
        lat = data['response']['groups'][0]["items"][0]["venue"]["location"]["lat"]
        long = data['response']['groups'][0]["items"][0]["venue"]["location"]["lng"]
        coords = (lat,long)
        return coords
    except IndexError:
        return None

    
amenities_coords = {"office": (best_office["lat"],best_office["long"])}
amenities = ["starbucks", "airport", "school", "club", "vegan", "basketball stadium", "hairdresser"]

for amenity in amenities:
    to_add = get_place_coords(best_office["geopoint"], amenity)
    amenities_coords[amenity] = to_add




In [29]:
map_4 = Map(location=[best_office["lat"],best_office["long"]],zoom_start=13)
map_4

for key,value in amenities_coords.items():
    name = {
        "location" : [value[0], value[1]],
        "tooltip" : key
    }
    
    Marker (**name).add_to(map_4)
map_4



**Best office coordinates**

In [27]:
best_office["lat"],best_office["long"]

(42.364014, -71.101375)